# 03. 합성곱 신경망

## 코드부터 작성하기

구현 모델: **합성곱 신경망(CNN)**을 이용한 28x28 흑백 이미지를 입력받아 셔츠, 바지, 드레스 등의 종류 구별

### 모듈 불러오기

In [ ]:
import tensorflow as tf

### 데이터 준비하기

In [ ]:
data = tf.keras.datasets.fashion_mnist
(training_images, training_labels), (test_images, test_labels) = data.load_data()

TensorFlow의 합성곱 신경망은 R,G,B 3가지의 채널이 존재하는 이미지에 대해 설계되었기 때문에 채널 차원(dimenssion)을 추가하여야한다.

- [`ndarray.reshape(shape, order='C')`](https://numpy.org/doc/stable/reference/generated/numpy.ndarray.reshape.html)

ndarray.reshape 메소드를 통해 기존 3차원(이미지 개수, 가로 픽셀, 세로 픽셀)이었던 데이터를 4차원(이미지 개수, 가로 픽셀, 세로 픽셀, 채널)으로 데이터를 재정의해준다.

기존 데이터는 흑백이므로 채널은 1개만 존재한다.

0-255의 값을0-1로 정규화하는 작업도 진행해준다.

In [ ]:
training_images = training_images.reshape(60000, 28, 28, 1)
training_images = training_images / 255.0
test_images = test_images.reshape(10000, 28, 28, 1)
test_images = test_images / 255.0

### 모델 구성하기

레이어 구성:
- 입력 (28,28,1) </br>
-> Conv2D(64개의 (3x3)필터 채널) + Relu </br>
Input: $(28, 28, 1)$ </br>
Output: $(26, 26, 64)$, 이미지 4면의 끝쪽 픽셀이 잘림 </br>
Params: $640=(inputChannels\times filtersize+bias)\times channels=(1\times3\times3+1)\times64$
-> MaxPooling2D(2x2 픽셀을 1x1로 압축) </br>
Input: $(26, 26, 64)$ </br>
Output: $(13, 13, 64)$</br>
-> Conv2D(64개의 (3x3)필터 채널) + Relu </br>
Input: $(13, 13, 64)$ </br>
Output: $(11, 11, 64)$, 이미지 4면의 끝쪽 픽셀이 잘림 </br>
Params: $36928=(inputChannels\times filtersize+bias)\times channels=(64\times3\times3+1)\times64$
-> MaxPooling2D(2x2 픽셀을 1x1로 압축)
Input: $(11, 11, 64)$ </br>
Output: $(5, 5, 64)$, </br>
-> Flatten(입력된 3차원 데이터를 1차원 배열로 재구성), 784 </br>
Input: $(5, 5, 64)$ </br>
Output: $1600=5\times5\times64$</br>
-> Dense(입력된 배열을 연산 후 128 길이의 배열로 출력) + Relu, 128 </br>
Input: $1600$ </br>
Output: $128$ </br>
-> Dense(입력된 배열을 연산 후 softmax를 통해 10개의 확률로 출력) + SoftMax, 10 </br>
Input: $128$ </br>
Output: $10$ </br>
- 출력(10)


손실 계산:
*   **옵티마이저(optimizer)**: Adam(Adaptive Moment Estimation)
*   **손실함수(loss, Loss Function)**: sparse_categorical_crossentropy(0,1,2,3 등의 정수 값의 라벨에 대한 다중 분류 손실함수)

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', input_shape=(28,28,1)),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

### 모델 학습하기
tf.keras.Model.fit(x, y)


*   Input = training_images
*   ouput = training_labels
*   epochs = 학습의 반복 수

In [ ]:
model.fit(training_images, training_labels, epochs=50)

Epoch 1/50
1875/1875 [==============================] - 86s 44ms/step - loss: 0.4433 - accuracy: 0.8403
Epoch 2/50
1875/1875 [==============================] - 79s 42ms/step - loss: 0.2956 - accuracy: 0.8911
Epoch 3/50
1875/1875 [==============================] - 81s 43ms/step - loss: 0.2486 - accuracy: 0.9079
Epoch 4/50
1875/1875 [==============================] - 80s 42ms/step - loss: 0.2147 - accuracy: 0.9205
Epoch 5/50
1875/1875 [==============================] - 77s 41ms/step - loss: 0.1874 - accuracy: 0.9310
Epoch 6/50
1875/1875 [==============================] - 79s 42ms/step - loss: 0.1636 - accuracy: 0.9381
Epoch 7/50
1875/1875 [==============================] - 78s 42ms/step - loss: 0.1415 - accuracy: 0.9473
Epoch 8/50
1875/1875 [==============================] - 81s 43ms/step - loss: 0.1248 - accuracy: 0.9529
Epoch 9/50
1875/1875 [==============================] - 79s 42ms/step - loss: 0.1085 - accuracy: 0.9585
Epoch 10/50
1875/1875 [==============================] - 76s 40m

### 예측하기

In [ ]:
model.evaluate(test_images, test_labels)

313/313 [==============================] - 4s 11ms/step - loss: 1.0087 - accuracy: 0.9072


[1.0086950063705444, 0.9071999788284302]


* loss(error, 손실) = 1.0087
* accuracy(정확도) = 90.72%

## 모델 출력 살펴보기

In [ ]:
classifications = model.predict(test_images)
print(classifications[0])
print(test_labels[0])

313/313 [==============================] - 3s 11ms/step
[0.0000000e+00 2.4038821e-33 2.5273516e-33 4.5973653e-35 4.2456518e-24
 4.0134513e-31 0.0000000e+00 6.5647037e-20 8.0693845e-38 9.9999994e-01]
9


모델은 라벨 0-9 에대한 확률을 출력한다. 라벨 9의 확률이 99.9%로 가장 높아 실제 정답인 9와 일치한다.

완전 연결 계층으로만 이루어진 모델로 학습하였을 경우 86%였던 정확도가 90.72%로 크게 오른 모습을 보인다. 레이어 구조를 더 강화하고 에폭 수를 조절하면 더 좋은 결과를 보일 것으로 예측된다.